In [1]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('/content/email_spam_classification.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,email,spam_or_not_spam
0,Lunch on Friday?,not spam
1,"Hello, how are you?",not spam
2,You won't believe what happened,spam
3,Important update about your account,not spam
4,Congratulations on your win!,spam


In [9]:
INTRO_BLURB = "Below is an instruction that describes a task. Write response that email is spam or not."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Sentence:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"



PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [10]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['email', 'spam_or_not_spam'],
    num_rows: 20
})

In [11]:
def apply_prompt_template(examples):
  instruction = "Find the spam or not spam for given email"
  context = examples.get("email")
  response = examples["spam_or_not_spam"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)

  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [12]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

Sentence:
Lunch on Friday?

### Response:
not spam

### End


In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [15]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 26
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)


In [16]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [17]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [18]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.892300
2,2.590300
3,2.325000
4,1.909600
5,1.507000
6,1.078100
7,0.836800
8,0.659000
9,0.486200
10,0.404800


TrainOutput(global_step=26, training_loss=0.6744499011681631, metrics={'train_runtime': 147.6301, 'train_samples_per_second': 2.818, 'train_steps_per_second': 0.176, 'total_flos': 586645847212032.0, 'train_loss': 0.6744499011681631, 'epoch': 20.8})

**Testing**

In [19]:
from transformers import pipeline

In [20]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [21]:
prompt = f"""
Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
Win a gift card worth $1000.

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
Win a gift card worth $1000.

### Response:
spam


None


In [22]:
prompt = f"""
Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
Please participate this meeting.

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
Please participate this meeting.

### Response:
not spam


None


In [23]:
prompt = f"""
Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
You get reward points.

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write response that email is spam or not.

### Instruction:
Find the spam or not spam for given email

email:
You get reward points.

### Response:
spam


None
